In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string as st
import re
import nltk
from nltk import PorterStemmer, WordNetLemmatizer
import nltk.corpus
import os

In [ ]:
nltk.download('all')

In [ ]:
from nltk.corpus import shakespeare
print(shakespeare.fileids())
for py_text in shakespeare.fileids ():
    print(py_text, shakespeare.words (py_text)[:20])

['a_and_c.xml', 'dream.xml', 'hamlet.xml', 'j_caesar.xml', 'macbeth.xml', 'merchant.xml', 'othello.xml', 'r_and_j.xml']
a_and_c.xml ['The', 'Tragedy', 'of', 'Antony', 'and', 'Cleopatra', 'Dramatis', 'Personae', 'MARK', 'ANTONY', 'OCTAVIUS', 'CAESAR', 'M', '.', 'AEMILIUS', 'LEPIDUS', 'triumvirs', '.', 'SEXTUS', 'POMPEIUS']
dream.xml ['A', 'Midsummer', 'Night', "'", 's', 'Dream', 'Dramatis', 'Personae', 'THESEUS', ',', 'Duke', 'of', 'Athens', '.', 'EGEUS', ',', 'father', 'to', 'Hermia', '.']
hamlet.xml ['The', 'Tragedy', 'of', 'Hamlet', ',', 'Prince', 'of', 'Denmark', 'Dramatis', 'Personae', 'CLAUDIUS', ',', 'king', 'of', 'Denmark', '.', 'HAMLET', ',', 'son', 'to']
j_caesar.xml ['The', 'Tragedy', 'of', 'Julius', 'Caesar', 'Dramatis', 'Personae', 'JULIUS', 'CAESAR', 'OCTAVIUS', 'CAESAR', 'MARCUS', 'ANTONIUS', 'M', '.', 'AEMILIUS', 'LEPIDUS', 'triumvirs', 'after', 'death']
macbeth.xml ['The', 'Tragedy', 'of', 'Macbeth', 'Dramatis', 'Personae', 'DUNCAN', ',', 'king', 'of', 'Scotland', '.', 

In [ ]:
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from collections import Counter
def text_lowercase(text):
  return text.lower()
def remove_punctuation(text):
  translator = str.maketrans('', '', string.punctuation)
  return text.translate(translator)
def remove_stopwords(text):
  stop_words = set(stopwords.words("english"))
  word_tokens = word_tokenize(text)
  filtered_text = [word for word in word_tokens if word not in stop_words]
  return filtered_text
def word_count(Words):
  my_counter = Counter()
  for word in Words:
      my_counter.update(word.split())
  return my_counter

In [ ]:
def preprocess(play):
  sent=[]
  sentences=[]
  for p in play:
      sent.append(list(p.itertext()))
  sent = sent[4:]
  for i in sent:
    for j in i:
      if '\n' not in j:
        sentences.append(j)
  newfile=[]
  lowercase_text=[]
  wo_punc=[]
  wo_sw=[]
  for i in range(len(sentences)):
    text = sentences[i]
    lowercase_text.append(text.lower())
    wo_punc.append(remove_punctuation(lowercase_text[i]))         #Removing punctuation from output text
    wo_sw.append(remove_stopwords(wo_punc[i]))                    #Removing stopwords from output text
  return wo_sw

In [ ]:
def wordlistbow(nfile):
  wordlist=[]
  for i in range(len(nfile)):
    for j in range(len(nfile[i])):
      if nfile[i][j] not in wordlist:
        wordlist.append(nfile[i][j])
  return wordlist

In [ ]:
def bag_of_words(play):
  newfile=preprocess(play)
  wordlist=wordlistbow(newfile)
  bow=[]
  for i in range(len(newfile)):
    bowsent=np.zeros(len(wordlist)).astype(int)
    for j in range(len(newfile[i])):
      ind=wordlist.index(newfile[i][j])
      bowsent[ind]+=1
    bow.append(bowsent)
  return bow

In [ ]:
bow1 = bag_of_words(shakespeare.xml('a_and_c.xml'))
bow1[:5]

[array([1, 0, 0, ..., 0, 0, 0]),
 array([0, 1, 1, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0])]

In [ ]:
def wordlisttfidf(nfile):
  wordlist=[]
  wordcount=dict()
  for i in range(len(nfile)):
    for j in range(len(nfile[i])):
      w=nfile[i][j]
      if w not in wordlist:
        wordlist.append(w)
        wordcount.update({w:1})
      else:
        c=wordcount[w]
        wordcount.update({w:c+1})
  return wordlist,wordcount

In [ ]:
def termfreq(term,play):
  newfile=preprocess(play)
  wordlist,wordcount=wordlisttfidf(newfile)
  tot_words=0
  for i in wordcount.values():
    tot_words+=i
  # tf=dict()
  # for i in wordcount.keys():
  #   count=wordcount[i]
  #   t=count/tot_words
  #   tf.update({wordcount:t})
  return wordcount[term]/tot_words

In [ ]:
print(termfreq('palace',shakespeare.xml('a_and_c.xml')))

0.0007486558225005105


In [ ]:
def tf_idf(term,playname,*play):
  wordlists=[]
  for i in range(len(play)):
    newfile=preprocess(play[i])
    wordlists.append(wordlisttfidf(newfile)[0])
  tot_docs=len(wordlists)
  no_doc_w_term=0
  for i in wordlists:
    if term in i:
      no_doc_w_term+=1
  tfidf_val = termfreq(term,play[0])*np.log(tot_docs/no_doc_w_term)
  print(f"The TF-IDF value for {term} in {playname} is {tfidf_val}")
  return tfidf_val

In [ ]:
play1='a_and_c.xml'
tf_idf('alexandria',play1,shakespeare.xml(play1),shakespeare.xml('dream.xml'))

The TF-IDF value for alexandria in a_and_c.xml is 0.0008963313435403907


0.0008963313435403907